# Общий код между ноутбуками

In [1]:
from sklearn.model_selection import train_test_split

# Дискретизация

In [ ]:
def entropy(a):
    vc = a.value_counts()
    vc/=vc.sum()
    return -(vc*np.log(vc)).sum()

def discretize(data, variables, icat, icont, contdiscstrategy="kmeans", n_bins=3):
    transformers_data = dict()
    
    if icat is None: # в датасете только непрерывные фичи (и предиктор)
        pipeline = make_pipeline(
            KBinsDiscretizer(n_bins=n_bins, encode="ordinal", random_state=42, strategy=contdiscstrategy), 
            FunctionTransformer(lambda x: x.astype("int")))
    elif icont is None: # в датасете только категориальные фичи (и предиктор)
        pipeline = make_pipeline(
            OrdinalEncoder(categories="auto"), 
            FunctionTransformer(lambda x: x.astype("int")))
    else:
        pipeline = make_pipeline(make_union(
        make_pipeline(FunctionTransformer(lambda x: x.iloc[:, icat]), OrdinalEncoder(categories="auto")),
        make_pipeline(FunctionTransformer(lambda x: x.loc[:, icont]), KBinsDiscretizer(n_bins=n_bins, encode="ordinal", random_state=42, strategy=contdiscstrategy))
    ),
        FunctionTransformer(lambda x: x.astype("int")))
    
    
    pipeline.fit(data)
    encoded_data = pd.DataFrame(pipeline.transform(data), columns=data.columns if icat is None or icont is None\
                                                                                        else data.columns[icat+icont])
    encoded_data = encoded_data[variables] # в общем случае пайплайн переставляет признаки, возвращаем их на их места здесь

    for feat in encoded_data.columns:
        if entropy(encoded_data[feat]) < 0.5:
            print(f"Warning: feature {feat} has practically degenerate states and low entropy")
    transformers_data["transformer"] = pipeline
    
    if icat is None:
        transformers_data["cont_features"]       = list(pipeline.steps[0][1].get_feature_names_out())
        transformers_data["cont_features_edges"] = pipeline.steps[0][1].bin_edges_
        
    elif icont is None:
        transformers_data["cat_features"]            = list(pipeline.steps[0][1].get_feature_names_out())
        transformers_data["cat_features_categories"] = pipeline.steps[0][1].categories_
        
    else:
        transformers_data["cont_features"]           = list(pipeline.steps[0][1].named_transformers["pipeline-2"].steps[1][1].get_feature_names_out())
        transformers_data["cont_features_edges"]     = pipeline.steps[0][1].named_transformers["pipeline-2"].steps[1][1].bin_edges_
        transformers_data["cat_features"]            = list(pipeline.steps[0][1].named_transformers["pipeline-1"].steps[1][1].get_feature_names_out())
        transformers_data["cat_features_categories"] = pipeline.steps[0][1].named_transformers["pipeline-1"].steps[1][1].categories_
    
    #kmeanspipeline.steps[0][1].named_transformers["pipeline-1"].steps[1][1].categories_[0]
    return encoded_data, transformers_data

#disc_data, pipeline_data = discretize(data, variables, icat=None, icont=[0, 1])


## Сборка таблиц усл. вероятностей по БС с градациями

In [ ]:
def collect_all_cpds(bn_info, distributions, n_states_map):
    cpds = list()
    for index, row in bn_info.iterrows():
        feat = row["name"].name
        if len(row["parents"]) == 0:
            # cpd is just a pd
            cpd = TabularCPD(feat, n_states_map[feat], [[e] for e in distributions[feat]["cprob"]])
            cpds.append(cpd)
        else:
            cpd_list = [probs for probs in distributions[feat]["cprob"].values()]
            #cpd_list = [probs for i, probs in distributions[feat]["cprob"].items() if i[0]!="["]
            #print(cpd_list)
            nrows = len(cpd_list)
            ncols = len(cpd_list[0])
            cpd_list = [[cpd_list[i][j] for i in range(nrows)] for j in range(ncols)]
            #print(feat, row["parents"])
            #print(cpd_list)
            cpd = TabularCPD(feat, n_states_map[feat], cpd_list, evidence=row["parents"], evidence_card=[n_states_map[p] for p in row["parents"]])
            cpds.append(cpd)
    return cpds

In [ ]:
def bootstrap_sampling(data,
                       d_dict,# словарь списков фактор-градация
                       b_sample_size,
                       metrics_list, trials=1000, alpha=0, incl_random_removal=False,
                       mode='regr',
                       drop_mode='random',
                       reference='bn',
                       incl_test=False,
                       test_size=0.3,
                       stratify_tts=True):
    
    n_ref=np.inf

    metrics_results={
        k: [list() for _ in range(len(metrics_list))] for k in d_dict.keys()
    }
    metrics_results['init'] = [list() for _ in range(len(metrics_list))]

    

    if incl_random_removal:
        metrics_results['random'] = [list() for _ in range(len(metrics_list))]

    metrics_results_test = {k: [list() for _ in range(len(metrics_list))] for k in metrics_results.keys()}

    n_dropped_stats={k: list() for k in metrics_results.keys()}

    #пробуем побутсрапировать выборку, чтобы оценить значимость различий в ошибках регрессии
    for i in tqdm(range(trials)):
        r_seed = 42+i
        np.random.seed(r_seed)
        #indexes = np.random.choice(data.index, size=b_sample_size)
        #bsample = data.loc[indexes].reset_index()[data.columns]
        if not incl_test:
            bsample = data.sample(n=b_sample_size, replace=False).reset_index()[data.columns]
        else:
            bsample_train, b_sample_test = train_test_split(data, random_state=r_seed,
                                                            test_size=test_size,
                                                            stratify=data[target] if stratify_tts else None)
            bsample = bsample_train.sample(n=b_sample_size, replace=True).reset_index()[data.columns]
        
        model_base = copy(model)
        model_base.fit(bsample[features], bsample[target])

        if mode=='regr':
            y_pred = model_base.predict(bsample[features]) # depends on task (regr/classif)
        else:
            y_pred = model_base.predict_proba(bsample[features])[:, 1] # depends on task (regr/classif)

        data_errors = bsample.copy(deep=True)
        if mode=='regr':
            data_errors["ape_error"] = np.abs((bsample[target]-y_pred)/bsample[target])
        else:
            #inconf scode
            data_errors["inconf_error"]=(1-y_pred)*bsample[target]+(y_pred)*(1-bsample[target])


        data_errors.drop(columns=[target], inplace=True)

        bsample_disc = pd.DataFrame(pipeline_data["transformer"].transform(data_errors), columns=data_errors.columns)
        
        bsample_disc = bsample_disc.reindex(index=data_errors.index)
        
        #print(pipeline_data["transformer"].transform(data_errors).shape[0])
        iterlist = [reference]+[x for x in metrics_results.keys() if x != reference]

        for g in iterlist:
            mask = False
            if g == 'init':
                mask = None
                samp = bsample
            elif g == 'random':
                mask=None
                indexes_to_drop = np.random.choice(bsample.index, size=n_ref, replace=False)
                samp = bsample.drop(index=indexes_to_drop)
            else:
                mask=False
                for feat, cat in zip(d_dict[g][0], d_dict[g][1]):
                    mask = mask | (bsample_disc[feat]==cat)
                
                if drop_mode == 'random':
                    indexes_to_drop = np.random.choice(bsample[mask].index, size=int(np.floor(bsample[mask].shape[0]*(1-alpha))), replace=False)
                elif drop_mode == 'metric':
                    if mode=='regr':
                        errors = data_errors[mask]['ape_error']
                    else:
                        #inconf scode
                        #print(g)
                        #print(mask.index, data_errors.index)
                        #print(bsample.shape[0], bsample_disc.shape[0], data_errors.shape[0], mask.shape, indexes.shape[0])
                        errors = data_errors[mask]["inconf_error"]

                    errors_sorted = errors.sort_values(ascending=False)
                    size = min(n_ref, int(np.floor(errors.shape[0]*(1-alpha))))

                    # т.к. подходы могут отбрасывать разное кол-во наблюдений, получаемый результат по метрикам будет разным из-за этого

                    #thresh = errors_sorted[:size].min()
                    #indexes_to_drop = errors[errors>=thresh].index
                    thresh = errors_sorted[-size:].max()
                    indexes_to_drop = errors[errors<thresh].index
                
                samp = bsample.drop(index=indexes_to_drop)
                if g==reference:
                    n_ref=indexes_to_drop.shape[0]
                    
                n_dropped_stats[g].append(indexes_to_drop.shape[0])
        # --
            
            X1, y1 = samp[features], samp[target]

            model1 = copy(model)
            model1.fit(X1, y1)  

            def calculate_metrics(model, X_data, y_data):
                dict_m = {}
                if mode=='regr':
                    y1_pred = model.predict(X_data) # depends on task (regr/classif)
                else:
                    y1_pred = model.predict_proba(X_data)[:, 1] # depends on task (regr/classif)

                for k, metric in enumerate(metrics_list):
                    dict_m[k] = metric(y_data, y1_pred)

                return dict_m



            train_res = calculate_metrics(model1, X1, y1)
            for k, metric in enumerate(metrics_list):
                metrics_results[g][k].append(train_res[k])

            if incl_test:
                test_res = calculate_metrics(model1, b_sample_test[features], b_sample_test[target])
                for k, metric in enumerate(metrics_list):
                    metrics_results_test[g][k].append(test_res[k])

    return_value = {'metrics': metrics_results,
             'n_dropped': n_dropped_stats,
             'avg_n_dropped': {k: sum(v)/len(v) if len(v) >0 else -1 for k, v in n_dropped_stats.items()}}
    
    if incl_test:
        return_value['metrics_test'] = metrics_results_test

    return return_value